# IDEAS
1. If the custom reading speed for a book is set, take that for calculating the number of days or take the generic one asked during program execution
2. If the custom number of times to read for a book is set, take that for calculating the number of days or take the generic one asked during program execution

# TODOS
1. Remove the need to populate the excel file with dates. Add the dates from today or a custom date.
2. Take the input of reading speed while running the program
3. Add the column of reading speed in excel to set custom reading speed for each book
4. Add the column of number of times to read a book - prompt for an input while starting program execution

In [6]:
import pandas as pd
import numpy as np
import datetime

In [12]:
today = datetime.date.today()
print(today.isoweekday())

7


In [13]:
tomorrow = today + datetime.timedelta(days=1)
print(tomorrow)

2022-08-01


In [14]:
#todo abstract the implementation of the while with an iterator for the same
list_of_dates = [today]
while tomorrow < datetime.date(2023, 1, 1):
    list_of_dates.append(tomorrow)
    tomorrow += datetime.timedelta(days=1)
print(len(list_of_dates))

154


In [118]:
days = pd.read_excel('Days.xlsx', sheet_name = 'Days', parse_dates=['Dates'])

In [119]:
days.head()

,Dates,Weekdays
0,2021-10-15,Friday
1,2021-10-16,Saturday
2,2021-10-17,Sunday
3,2021-10-18,Monday
4,2021-10-19,Tuesday


In [120]:
books = pd.read_excel('Days.xlsx', sheet_name='Books')
books.head()

,Name,Class,Pages
0,Geography of India,10.0,166
1,Economic Geography,10.0,144
2,Principles of Geography - 1,11.0,195
3,Principles of Geography - 2,11.0,148
4,India: A Generational Geography,12.0,161


In [121]:
workdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
weekends = ['Saturday', 'Sunday']

In [122]:
# Reading speed to be added.
def pages(day: str): 
    if day in workdays:
        return 30
    return 60

In [123]:
pages(days.iloc[[0]].Weekdays[0])

30

In [124]:
days.dtypes

Dates       datetime64[ns]
Weekdays            object
dtype: object

In [125]:
days['Target'] = 0
days['Book'] = ''
days['Class'] = 0
days.head()

,Dates,Weekdays,Target,Book,Class
0,2021-10-15,Friday,0,,0
1,2021-10-16,Saturday,0,,0
2,2021-10-17,Sunday,0,,0
3,2021-10-18,Monday,0,,0
4,2021-10-19,Tuesday,0,,0


In [126]:
currDay = 0
targetPerDay = []
bookPerDay = []
classPerDay = []
for bookNumber in books.index:
    book = books.loc[bookNumber]
    pagesRemaining = book.Pages
    # print(type(pagesRemaining))
    # print(pagesRemaining)
    # print(pagesRemaining - 20)
    # break
    while pagesRemaining > 0 and currDay <= len(days):
        day = days.loc[currDay]
        targetPerDay.append(pages(day.Weekdays) if pages(day.Weekdays) < pagesRemaining else pagesRemaining)
        bookPerDay.append(book.Name)
        classPerDay.append(book.Class)
        pagesRemaining -= targetPerDay[-1]
        currDay += 1
    if pagesRemaining != 0:
        print('Cannot allocate all books. Increase the number of pages covered in each day')
        break
else:
    print('Allocated Successfully!')
    print('Total Pages =', sum(targetPerDay))
    print('Total Days Required at Current Speed =', len(targetPerDay))
    
    

Allocated Successfully!
Total Pages = 5116
Total Days Required at Current Speed = 143


In [127]:
if len(targetPerDay) < len(days):
    diff = len(days) - len(targetPerDay)
    for item in [targetPerDay, bookPerDay, classPerDay]:
        item.extend([None]*diff)

In [128]:
print(len(targetPerDay), len(bookPerDay), len(classPerDay))

229 229 229


In [129]:
days.Target = targetPerDay
days.Book = bookPerDay
days.Class = classPerDay
days.head()

,Dates,Weekdays,Target,Book,Class
0,2021-10-15,Friday,30.0,Geography of India,10.0
1,2021-10-16,Saturday,60.0,Geography of India,10.0
2,2021-10-17,Sunday,60.0,Geography of India,10.0
3,2021-10-18,Monday,16.0,Geography of India,10.0
4,2021-10-19,Tuesday,30.0,Economic Geography,10.0


In [137]:
days.Dates = days.Dates.apply(lambda x: x.strftime('%d-%m-%Y'))

In [138]:
days.head()

,Dates,Weekdays,Target,Book,Class
0,15-10-2021,Friday,30.0,Geography of India,10.0
1,16-10-2021,Saturday,60.0,Geography of India,10.0
2,17-10-2021,Sunday,60.0,Geography of India,10.0
3,18-10-2021,Monday,16.0,Geography of India,10.0
4,19-10-2021,Tuesday,30.0,Economic Geography,10.0


In [141]:
with pd.ExcelWriter('Days.xlsx', mode='a', if_sheet_exists='replace') as writer:
    days.to_excel(writer, sheet_name='Plan')